# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
csv_file = os.path.join('..', 'WeatherPy', 'Output', 'city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3874787,Punta Arenas,CL,-53.15,-70.92,42.39,70,40,2.24
1,1817240,Baiyin,CN,36.56,104.21,51.80,29,40,11.18
2,3369157,Cape Town,ZA,-33.93,18.42,64.85,52,20,4.70
3,2071860,Esperance,AU,-33.87,121.90,71.60,46,0,10.29
4,1015776,Bredasdorp,ZA,-34.53,20.04,68.00,45,99,5.82
...,...,...,...,...,...,...,...,...,...
548,8079926,Labuhan,ID,-6.88,112.21,85.75,72,17,8.41
549,4705086,Lamar,US,33.67,-95.58,54.82,93,1,6.93
550,1712961,Gigmoto,PH,13.78,124.39,83.73,78,38,10.89
551,2779229,Friesach,AT,46.96,14.41,47.03,57,20,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)


locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
231,2450173,Taoudenni,ML,22.68,-3.98,84.13,14,0,5.93
305,3598787,Champerico,GT,14.30,-91.92,81.73,69,3,3.96
365,962367,Richards Bay,ZA,-28.78,32.04,76.39,61,0,6.49
371,989921,Klerksdorp,ZA,-26.85,26.67,78.08,17,0,4.34
385,1529626,Baijiantan,CN,45.63,85.18,77.67,16,0,6.04
457,3592086,Ocós,GT,14.51,-92.19,81.82,69,1,3.56
530,2079582,Alyangula,AU,-13.85,136.42,89.60,62,0,4.70
536,3347019,Mossamedes,AO,-15.20,12.15,78.78,63,7,2.37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
231,2450173,Taoudenni,ML,22.68,-3.98,84.13,14,0,5.93,
305,3598787,Champerico,GT,14.30,-91.92,81.73,69,3,3.96,
365,962367,Richards Bay,ZA,-28.78,32.04,76.39,61,0,6.49,
371,989921,Klerksdorp,ZA,-26.85,26.67,78.08,17,0,4.34,
385,1529626,Baijiantan,CN,45.63,85.18,77.67,16,0,6.04,
457,3592086,Ocós,GT,14.51,-92.19,81.82,69,1,3.56,
530,2079582,Alyangula,AU,-13.85,136.42,89.60,62,0,4.70,
536,3347019,Mossamedes,AO,-15.20,12.15,78.78,63,7,2.37,


In [9]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
231,2450173,Taoudenni,ML,22.68,-3.98,84.13,14,0,5.93,NaN
305,3598787,Champerico,GT,14.30,-91.92,81.73,69,3,3.96,"Hotel y restaurante ""El Diamante"""
365,962367,Richards Bay,ZA,-28.78,32.04,76.39,61,0,6.49,Splendid Inn Bayshore
371,989921,Klerksdorp,ZA,-26.85,26.67,78.08,17,0,4.34,Bateleur Guesthouse
385,1529626,Baijiantan,CN,45.63,85.18,77.67,16,0,6.04,NaN
457,3592086,Ocós,GT,14.51,-92.19,81.82,69,1,3.56,NaN
530,2079582,Alyangula,AU,-13.85,136.42,89.60,62,0,4.70,Groote Eylandt Lodge
536,3347019,Mossamedes,AO,-15.20,12.15,78.78,63,7,2.37,Hotel ibis Styles Iu Namibe


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))